# Trainingspipeline 20

- ohne Bigramme
- vector_size: 300
- window: 5
- min_count: 1
- seed: 1

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
print(stopwords)

['ad', 'al', 'allo', 'ai', 'agli', 'all', 'agl', 'alla', 'alle', 'con', 'col', 'coi', 'da', 'dal', 'dallo', 'dai', 'dagli', 'dall', 'dagl', 'dalla', 'dalle', 'di', 'del', 'dello', 'dei', 'degli', 'dell', 'degl', 'della', 'delle', 'in', 'nel', 'nello', 'nei', 'negli', 'nell', 'negl', 'nella', 'nelle', 'su', 'sul', 'sullo', 'sui', 'sugli', 'sull', 'sugl', 'sulla', 'sulle', 'per', 'tra', 'contro', 'io', 'tu', 'lui', 'lei', 'noi', 'voi', 'loro', 'mio', 'mia', 'miei', 'mie', 'tuo', 'tua', 'tuoi', 'tue', 'suo', 'sua', 'suoi', 'sue', 'nostro', 'nostra', 'nostri', 'nostre', 'vostro', 'vostra', 'vostri', 'vostre', 'mi', 'ti', 'ci', 'vi', 'lo', 'la', 'li', 'le', 'gli', 'ne', 'il', 'un', 'uno', 'una', 'ma', 'ed', 'se', 'perché', 'anche', 'come', 'dov', 'dove', 'che', 'chi', 'cui', 'non', 'più', 'quale', 'quanto', 'quanti', 'quanta', 'quante', 'quello', 'quelli', 'quella', 'quelle', 'questo', 'questi', 'questa', 'queste', 'si', 'tutto', 'tutti', 'a', 'c', 'e', 'i', 'l', 'o', 'ho', 'hai', 'ha', 'ab

In [4]:
df = pd.read_csv('../../Korpus/Korpus/corpus_lemmatized.csv', sep=',', encoding='utf-8')

In [5]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text
0,Poesia.IV.11.Testo.txt,MIDIA,Giambattista Felice Zappi,Poesie,1700.0,1700-1750,poesia,IV. 11. Rime degli arcadi: Tirsi Leucasio (Gio...,6113.0,iv . 11 . rima del arcade : tirsi leucasio ( g...
1,Personali.IV.5.Testo.txt,MIDIA,Lorenzo Magalotti,Lettere odorose (1693-1705),1700.0,1700-1750,personale,"IV. 5. Lorenzo Magalotti, Lettere odorose (169...",8374.0,"iv . 5 . Lorenzo magalotti , lettere odoroso (..."
2,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,"﻿iv . 4 . Ludovico Antonio muratori , antichit..."
3,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0,[ proemio ] \n prendere a scrivere il mio vita...
4,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0,"iv . 4 . Vincenzo da filicaia , lettere inedit..."


In [6]:
df.shape

(304129, 10)

In [7]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [8]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [9]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1985']
df10 = df_periods['1986-2000']
df11 = df_periods['2001-2021']

In [10]:
df9.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text
74265,diacorisBiagi1.txt,DiaCoris,Enzo Biagi,L'alluvione del Polesine,1951.0,1951-1985,stampa,L ' angoscia di un padre Si parla di battaglie...,152.0,l ' angoscia di uno padre si parlare di battag...
74266,LISCastiglia1.txt,LIS,Roderigo di Castiglia,«Vittorini se n'è ghiuto/ E soli ci ha lasciat...,1951.0,1951-1985,stampa,"A dire il vero , nelle nostre file pochi se ne...",1695.0,"a dire il vero , nella nostro file pochi se ne..."
74267,LISBianchi1.txt,LIS,Ranuccio Bianchi Bandinelli,Il nostro lavoro nella scuola,1951.0,1951-1985,stampa,"Come in tutti i congressi , anche nel VII Cong...",2898.0,"come in tutto il congresso , anche nel vii con..."
74268,LLAlbertelli1.txt,Liber Liber,Pilo Albertelli,Rousseau,1951.0,1951-1985,prosa letteraria,﻿Pilo Albertelli\nRousseau\n\n Nacque il 28 g...,4894.0,﻿pilo albertelli \n rousseau \n\n nascere il...
74269,LISJotti1.txt,LIS,Nilde Jotti,La questione dei fumetti,1951.0,1951-1985,stampa,Il dibattito sulla stampa a fumetti per i raga...,2785.0,il dibattito sulla stampa a fumetto per il rag...


## Training von Word2Vec

In [11]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [13]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 5                         # The maximum distance between the current and predicted word within a sentence
min_count = 1                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1

In [14]:
# Ordner anlegen zum Abspeichern von trainierten Modellen

if not os.path.exists('../trained_models'):
    os.makedirs('../trained_models')

### Zeitraum 1: 1700-1750

In [15]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [16]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 17.6 s


In [17]:
print(sentences1[3])

['lungo', 'inospita', 'campagna', 'aggiungere', 'invidia', 'lato', 'manco', 'dire', 'anch', 'essere', 'teco', 'labbro', 'bianco', 'veggo', 'veneno', 'cor', 'stagno']


In [18]:
len(sentences1)

113328

In [19]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 58s


In [20]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('sconfiggere', 0.7859934568405151),
 ('desolazione', 0.780498743057251),
 ('sacco', 0.7795285582542419),
 ('rumore', 0.7793293595314026),
 ('costernazione', 0.7699690461158752),
 ('riempiè', 0.7684776186943054),
 ('spavento', 0.7665495276451111),
 ('peste', 0.7642766237258911),
 ('macello', 0.763086199760437),
 ('torma', 0.7604858875274658)]

In [21]:
# trainiertes Modell speichern

w2v1.save(os.path.join('../trained_models/Word2Vec20', '20w2v1.model'))

### Zeitraum 2: 1751-1800

In [22]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [23]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 20.7 s


In [24]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 49s


In [25]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('letargo', 0.8964829444885254),
 ('varco', 0.8940438628196716),
 ('respingere', 0.8908490538597107),
 ('scherno', 0.889966607093811),
 ('colmo', 0.8884931802749634),
 ('fantasma', 0.887101948261261),
 ('ruina', 0.8856281638145447),
 ('usurpatore', 0.8839372992515564),
 ('sprone', 0.883405864238739),
 ('retaggio', 0.8831426501274109)]

In [26]:
w2v2.save(os.path.join('../trained_models/Word2Vec20', '20w2v2.model'))

### Zeitraum 3: 1801-1825

In [27]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [28]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 16.4 s


In [29]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 51s


In [30]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('turpe', 0.8612518310546875),
 ('avversa', 0.8523948192596436),
 ('infiammare', 0.8513959646224976),
 ('rinfrancare', 0.8452512621879578),
 ('rissa', 0.8443565368652344),
 ('orrore', 0.8438355922698975),
 ('vituperio', 0.8437812328338623),
 ('avvampare', 0.8431482911109924),
 ('furore', 0.8428069353103638),
 ('impotente', 0.8406247496604919)]

In [31]:
w2v3.save(os.path.join('../trained_models/Word2Vec20', '20w2v3.model'))

### Zeitraum 4: 1826-1850

In [32]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [33]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 16.4 s


In [34]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 52s


In [35]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('furore', 0.8491377234458923),
 ('opprimere', 0.8358274698257446),
 ('indegnazione', 0.8315457701683044),
 ('smisurato', 0.8273174166679382),
 ('sfrenato', 0.8239211440086365),
 ('furor', 0.8234388828277588),
 ('gelosia', 0.8214377760887146),
 ('frenare', 0.8137011528015137),
 ('sdegno', 0.813094973564148),
 ('servaggio', 0.810877799987793)]

In [36]:
w2v4.save(os.path.join('../trained_models/Word2Vec20', '20w2v4.model'))

### Zeitraum 5: 1851-1875

In [37]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [38]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 16.8 s


In [39]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 46s


In [40]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('sgomento', 0.8857079744338989),
 ('disperazione', 0.8782998323440552),
 ('fremito', 0.8756722211837769),
 ('assalire', 0.8710376620292664),
 ('esaltazione', 0.8659152388572693),
 ('sdegno', 0.8635143637657166),
 ('impeto', 0.8620864152908325),
 ('furore', 0.8614261746406555),
 ('atterrire', 0.8593610525131226),
 ('spavento', 0.8576532006263733)]

In [41]:
w2v5.save(os.path.join('../trained_models/Word2Vec20', '20w2v5.model'))

### Zeitraum 6: 1876-1900

In [42]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [43]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 17.8 s


In [44]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 44s


In [45]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('tremendo', 0.8208913803100586),
 ('sgomento', 0.8197522163391113),
 ('furore', 0.8174567222595215),
 ('spavento', 0.8136556148529053),
 ('parossismo', 0.806135892868042),
 ('orrore', 0.805837094783783),
 ('crisi', 0.8024734854698181),
 ('accesso', 0.8006179928779602),
 ('convulsione', 0.7991422414779663),
 ('violento', 0.797741711139679)]

In [46]:
w2v6.save(os.path.join('../trained_models/Word2Vec20', '20w2v6.model'))

### Zeitraum 7: 1901-1925

In [47]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [48]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 17.6 s


In [49]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 42s


In [50]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('indicibile', 0.8450005054473877),
 ('furore', 0.8355855941772461),
 ('indignazione', 0.8322768211364746),
 ('spavento', 0.8295801281929016),
 ('commozione', 0.8287470936775208),
 ('subitaneo', 0.8280487656593323),
 ('orrore', 0.8279702663421631),
 ('smarrimento', 0.8247280716896057),
 ('straziante', 0.8195492029190063),
 ('invocazione', 0.813363790512085)]

In [51]:
w2v7.save(os.path.join('../trained_models/Word2Vec20', '20w2v7.model'))

### Zeitraum 8: 1926-1950

In [52]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [53]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 16.9 s


In [54]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 43s


In [55]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.871465265750885),
 ('orrore', 0.85284423828125),
 ('raccapriccio', 0.8386695384979248),
 ('sussulto', 0.8377180695533752),
 ('ribrezzo', 0.8360605239868164),
 ('subitaneo', 0.8339870572090149),
 ('repentino', 0.8238382339477539),
 ('disgusto', 0.8218224048614502),
 ('disperazione', 0.8204032778739929),
 ('impeto', 0.8185206055641174)]

In [56]:
w2v8.save(os.path.join('../trained_models/Word2Vec20', '20w2v8.model'))

### Zeitraum 9: 1951-1985

In [57]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [58]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 18.1 s


In [59]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 45s


In [60]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('sconvolgere', 0.7701098918914795),
 ('orrore', 0.7662425637245178),
 ('sanguinoso', 0.7416909337043762),
 ('immane', 0.7406674027442932),
 ('seminare', 0.7350096106529236),
 ('annientare', 0.7332013249397278),
 ('panico', 0.7278039455413818),
 ('valanga', 0.7269976139068604),
 ('orribile', 0.7264589071273804),
 ('smarrimento', 0.7228153944015503)]

In [61]:
w2v9.save(os.path.join('../trained_models/Word2Vec20', '20w2v9.model'))

### Zeitraum 10: 1986-2000

In [62]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [63]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 36.7 s


In [64]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 47s


In [65]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('seminare', 0.8658721446990967),
 ('cieco', 0.8464635014533997),
 ('sanguinoso', 0.8413199782371521),
 ('insanguinare', 0.8394982814788818),
 ('spaventoso', 0.8345075845718384),
 ('lutto', 0.8317131400108337),
 ('rabbia', 0.825316846370697),
 ('ferita', 0.823995053768158),
 ('tremendo', 0.8232277631759644),
 ('disperazione', 0.8220595717430115)]

In [66]:
w2v10.save(os.path.join('../trained_models/Word2Vec20', '20w2v10.model'))

### Zeitraum 11: 2001-2021

In [67]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [68]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 15.4 s


In [69]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 9s


In [70]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('seminare', 0.9111881256103516),
 ('disperazione', 0.8982512950897217),
 ('barbaro', 0.8962597846984863),
 ('distruzione', 0.8941064476966858),
 ('sgomento', 0.8940295577049255),
 ('tormento', 0.8920628428459167),
 ('soffocare', 0.8893533945083618),
 ('dèi', 0.888884425163269),
 ('atrocemente', 0.8887830972671509),
 ('angoscia', 0.8886421322822571)]

In [71]:
w2v11.save(os.path.join('../trained_models/Word2Vec20', '20w2v11.model'))